In [39]:
import pandas as pd 
import numpy as np

df_psd = pd.read_csv("data/df_meg_psy_allfeatures.csv")
df_slopes = pd.read_csv("data/slopes_580features.csv")

df_slopes = df_slopes[df_slopes['target']==1].drop(columns='target')

In [41]:
df_psd.head(2)

,PATIENT_ID,MoCA corretto,LEDD Levodopa,LEDD_DA,LEDD totale,AGE,SEX,DD months,mH&Y,UPDRS III,...,Delta_channel114,Theta_channel114,Alpha_channel114,Beta_channel114,Gamma_channel114,Delta_channel115,Theta_channel115,Alpha_channel115,Beta_channel115,Gamma_channel115
0,BNTLGU53_01,20.98,150,0,150,63,0,24,2.0,29,...,40375.942038,24202.162437,37191.128954,7215.472327,1711.428597,48377.700893,28849.230923,43652.571297,8903.325322,2183.211469
1,BRRMRS48_01,14.40,300,0,400,70,1,24,2.0,21,...,12839.523331,5403.541702,7058.583578,2759.861741,1139.080707,20525.446929,8569.110052,10912.941664,4407.627448,1845.709506


In [43]:
df_slopes.head()

,Slope_Delta_channel1,Slope_Theta_channel1,Slope_Alpha_channel1,Slope_Beta_channel1,Slope_Gamma_channel1,Slope_Delta_channel2,Slope_Theta_channel2,Slope_Alpha_channel2,Slope_Beta_channel2,Slope_Gamma_channel2,...,Slope_Delta_channel115,Slope_Theta_channel115,Slope_Alpha_channel115,Slope_Beta_channel115,Slope_Gamma_channel115,Slope_Delta_channel116,Slope_Theta_channel116,Slope_Alpha_channel116,Slope_Beta_channel116,Slope_Gamma_channel116
20,-0.655566,0.142126,-0.843033,-0.875152,-10.003454,-0.692597,0.224640,-0.906742,-1.141044,-9.867734,...,-0.686671,0.090868,0.888668,-1.909757,-9.392111,-0.701925,0.042903,0.905722,-1.871405,-9.378056
21,-0.359439,0.367208,-2.740478,-1.716018,-9.740200,-0.326454,0.706182,-2.821907,-1.638603,-9.812921,...,-0.646110,1.135646,-3.535051,-2.086474,-9.405221,-0.588811,1.123969,-3.424744,-2.136706,-9.432555
22,-0.252845,0.624044,-1.780873,-1.010821,-10.242406,-0.247859,0.594507,-1.728943,-1.049204,-10.247460,...,-0.246354,1.048185,-2.146062,-1.556400,-9.860998,-0.253217,0.960149,-2.084448,-1.520070,-9.906138
23,-0.721164,-1.491600,-0.667739,-0.892241,-9.581164,-0.745927,-1.274181,-0.736438,-0.828379,-9.530035,...,-0.592974,-1.168283,0.605362,-1.653954,-9.538194,-0.642722,-1.259780,0.388393,-1.558507,-9.486133
24,-0.576592,-0.301414,-1.405497,-0.914081,-10.044762,-0.557758,-0.217200,-1.727260,-0.893627,-10.093848,...,-0.321660,-0.051064,-1.827930,-0.737840,-10.354136,-0.347705,-0.055564,-1.833280,-0.751383,-10.308971


In [45]:
print(df_psd.shape)
print(df_slopes.shape)

(20, 599)
(20, 580)


In [47]:
vector_bdi = df_psd["BDI"]
vector_apathy = df_psd["Apathy"]

In [49]:
thr1 = vector_bdi.quantile(0.5)+0.1  # This gives the median value
thr2 = vector_apathy.quantile(0.5)+0.1  # This gives the median value

bdi_binary = (vector_bdi >= thr1).astype(int)
apathy_binary = (vector_apathy >= thr2).astype(int)

print(bdi_binary.value_counts())
print(apathy_binary.value_counts())

BDI
0    11
1     9
Name: count, dtype: int64
Apathy
0    10
1    10
Name: count, dtype: int64


In [51]:
df_slopes = df_slopes.reset_index().drop(columns='index')

In [53]:
df_slopes_apathy = df_slopes.copy(deep=True)
df_slopes_bdi = df_slopes.copy(deep=True)
df_slopes_apathy['apathy_binary'] = apathy_binary
df_slopes_bdi['bdi_binary'] = bdi_binary

df_psd_apathy = df_psd.copy(deep=True)
df_psd_bdi = df_psd.copy(deep=True)
df_psd_apathy['apathy_binary'] = apathy_binary
df_psd_bdi['bdi_binary'] = bdi_binary

print(df_slopes_bdi.shape)
print(df_slopes_apathy.shape)

print(df_psd_apathy.shape)
print(df_psd_bdi.shape)

(20, 581)
(20, 581)
(20, 600)
(20, 600)


# Calculating Odds ratios 

In [56]:
df_psd_apathy = df_psd_apathy.iloc[:, 19:]
df_psd_bdi = df_psd_bdi.iloc[:, 19:]

In [58]:
print(df_slopes_bdi.shape)
print(df_slopes_apathy.shape)

print(df_psd_apathy.shape)
print(df_psd_bdi.shape)

(20, 581)
(20, 581)
(20, 581)
(20, 581)


In [126]:
def calc_odds(df, target_col):
    df_current = df.copy(deep=True)

    X = df_current.drop(columns=target_col)

    df_output = pd.DataFrame(index=X.columns, columns=['OR', 'log_OR', 'p-value', '2.5%', '97.5%'])
    
    for column in X.columns:
        model = smf.logit(f'{target_col} ~ {column}', data=df_current).fit()
        # Store results in the output DataFrame
        df_output.loc[column, 'OR'] = np.exp(model.params[1])
        df_output.loc[column, 'log_OR'] = model.params[1]
        df_output.loc[column, 'p-value'] = model.pvalues[1]
        conf_int = model.conf_int().loc[column]
        df_output.loc[column, '2.5%'] = np.exp(conf_int[0])
        df_output.loc[column, '97.5%'] = np.exp(conf_int[1])
        
    return df_output

In [128]:
df_slopes_bdi_odds = calc_odds(df_slopes_bdi, 'bdi_binary')
df_slopes_apathy_odds = calc_odds(df_slopes_apathy, 'apathy_binary')

Optimization terminated successfully.
         Current function value: 0.649153
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.687637
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.678979
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.666138
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.687448
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.622127
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.686671
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.665019
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.686678
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.687814
  

/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'OR'] = np.exp(model.params[1])
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'log_OR'] = model.params[1]
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

Optimization terminated successfully.
         Current function value: 0.687652
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683454
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.686135
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.647663
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.666685
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.688135
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.687193
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.684466
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.663038
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.683250
  

/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'OR'] = np.exp(model.params[1])
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'log_OR'] = model.params[1]
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

Optimization terminated successfully.
         Current function value: 0.687445
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.685433
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.687556
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.685422
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.657432
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.684808
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.679061
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.681393
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.646934
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.600063
  

...

/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'OR'] = np.exp(model.params[1])
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'log_OR'] = model.params[1]
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

Optimization terminated successfully.
         Current function value: 0.649655
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.599035
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.547793
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.666136
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.659572
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.675800
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.577432
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.596991
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.686484
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.666382
  

/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'OR'] = np.exp(model.params[1])
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'log_OR'] = model.params[1]
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

Optimization terminated successfully.
         Current function value: 0.690086
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692910
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.680400
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.663703
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.606387
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.673713
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.648388
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.686773
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.683565
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.679493
  

/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'OR'] = np.exp(model.params[1])
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'log_OR'] = model.params[1]
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

In [130]:
df_psd_bdi_odds = calc_odds(df_psd_bdi, 'bdi_binary')
df_psd_apathy_odds = calc_odds(df_psd_apathy, 'apathy_binary')

Optimization terminated successfully.
         Current function value: 0.385717
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.376018
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.413266
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.434791
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.511815
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.477270
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.438074
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.474561
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.443539
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.505978
  

/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'OR'] = np.exp(model.params[1])
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'log_OR'] = model.params[1]
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

Optimization terminated successfully.
         Current function value: 0.647503
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.658992
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.591365
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.590304
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.653535
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.657681
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.670331
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.640814
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.682668
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.653898
  

/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'OR'] = np.exp(model.params[1])
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'log_OR'] = model.params[1]
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

Optimization terminated successfully.
         Current function value: 0.674803
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.676697
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.683686
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.687577
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.668381
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.671643
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.679301
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.662588
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.646087
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.672659
  

...

/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'OR'] = np.exp(model.params[1])
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'log_OR'] = model.params[1]
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

Optimization terminated successfully.
         Current function value: 0.628404
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.603890
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.608155
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.607204
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.666151
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.644425
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.677108
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.665567
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.669484
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.671471
  

/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'OR'] = np.exp(model.params[1])
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'log_OR'] = model.params[1]
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

Optimization terminated successfully.
         Current function value: 0.680137
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.687187
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.668767
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.686430
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.685812
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.687479
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.672120
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.688490
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693116
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.683912
  

/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'OR'] = np.exp(model.params[1])
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_output.loc[column, 'log_OR'] = model.params[1]
/var/folders/lh/m93lk75s2gxcv8lhns7zq2_h0000gn/T/ipykernel_2746/497521636.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To a

In [132]:
df_slopes_bdi_odds.sort_values(by='OR', ascending=False, inplace=True)
df_slopes_apathy_odds.sort_values(by='OR', ascending=False, inplace=True)
df_psd_bdi_odds.sort_values(by='OR', ascending=False, inplace=True)
df_psd_apathy_odds.sort_values(by='OR', ascending=False, inplace=True)

In [140]:
df_slopes_bdi_odds.to_csv("../output/OR_slopes_bdi.csv", index=True)
df_slopes_apathy_odds.to_csv("../output/OR_slopes_apathy.csv", index=True)
df_psd_bdi_odds.to_csv("../output/OR_PSD_bdi.csv", index=True)
df_psd_apathy_odds.to_csv("../output/OR_PSD_apathy.csv", index=True)